In [2]:
import pandas as pd
import tqdm
import numpy as np

from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from sklearn.metrics.pairwise import cosine_similarity

In [3]:
online = pd.read_csv('01온라인행동정보.csv', encoding = 'utf-8')
trans = pd.read_csv('02거래정보.csv', encoding = 'utf-8')
product = pd.read_csv('04상품분류정보.csv', encoding = 'utf-8')

new_keyword = pd.read_csv('new_product.csv', header=None, encoding = 'utf-8')

C:\Users\Jaino\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### 구매기록이 있는 고객

In [4]:
trans_clnt_id = list(trans.clnt_id.unique())

In [5]:
y_clnt = online.query('clnt_id == @trans_clnt_id').sort_values(by = ['clnt_id','sess_id','hit_seq'])

In [6]:
search_trans = pd.merge(
    y_clnt, trans, how = 'left'
).loc[:,['clnt_id', 'sess_id','sech_kwd','pd_c']]

In [7]:
# 검색, 구매 둘다 NaN인 행 제거
y_df = search_trans.dropna(thresh=3).reset_index().iloc[:,1:];y_df.head()

,clnt_id,sess_id,sech_kwd,pd_c
0,2,1,레쓰비,NaN
1,2,1,레쓰비,NaN
2,2,1,레쓰비,NaN
3,2,1,레쓰비,NaN
4,2,1,레쓰비,NaN


In [8]:
product.head()

,pd_c,clac_nm1,clac_nm2,clac_nm3
0,1,Automotive Products,Automotive Replacement Repair / Maintanance Kits,Automobile Oil / Additives
1,2,Automotive Products,Automotive Replacement Repair / Maintanance Kits,Car Lights
2,3,Automotive Products,Automotive Replacement Repair / Maintanance Kits,Car Paint
3,4,Automotive Products,Automotive Replacement Repair / Maintanance Kits,Filters
4,5,Automotive Products,Automotive Replacement Repair / Maintanance Kits,Wiper Blades


In [9]:
def make_str(x):
    if x < 10:
        return '000' + str(x)
    elif x < 100:
        return '00' + str(x)
    elif x < 1000:
        return '0' + str(x)
    else:
        return str(x)
product['pd_c'] = product['pd_c'].apply(make_str)

In [10]:
p_c = product.clac_nm3.sort_values().dropna().reset_index().iloc[1:,1]
p_c2 = product.sort_values(by='clac_nm3').dropna().reset_index().iloc[1:,:]

In [11]:
keyword = pd.DataFrame({
    'clac_nm3': p_c.reset_index().clac_nm3,
    'kor_clac_nm3': new_keyword.iloc[:,0]
})

In [12]:
keyword

,clac_nm3,kor_clac_nm3
0,Accessory Bags,액세서리 가방
1,Accident Prevention Equipment,사고 방지 장비
2,Adhesive Tapes,접착 테이프
3,Adhesives,접착제
4,Adult's Bed Covers and Skirts,성인용 침대 커버와 스커트
...,...,...
1659,Young Radishes,어린 무
1660,Yuzus / Quinces,유주스 / 왕자들
1661,Zipper Poly Bags / Plastic Bags,지퍼 폴리 가방/플라스틱
1662,kelp,다시마


In [13]:
product = pd.merge(product, keyword, how = 'left');product

,pd_c,clac_nm1,clac_nm2,clac_nm3,kor_clac_nm3
0,0001,Automotive Products,Automotive Replacement Repair / Maintanance Kits,Automobile Oil / Additives,자동차 오일 / 첨가제
1,0002,Automotive Products,Automotive Replacement Repair / Maintanance Kits,Car Lights,카 라이트
2,0003,Automotive Products,Automotive Replacement Repair / Maintanance Kits,Car Paint,카 페인트
3,0004,Automotive Products,Automotive Replacement Repair / Maintanance Kits,Filters,필터
4,0005,Automotive Products,Automotive Replacement Repair / Maintanance Kits,Wiper Blades,와이퍼 블레이드
...,...,...,...,...,...
1668,1663,Women's Clothing,Women's Upper Bodywear / Tops,Women's Cardigans,여성 카디건
1669,1664,Women's Clothing,Women's Upper Bodywear / Tops,Women's Sweaters / Pullover,여자 스웨터/풀오버
1670,1665,Women's Clothing,Women's Upper Bodywear / Tops,Women's Sweatshirts / Hoodies,여성 스웨트 셔츠 / 후드티
1671,1666,Women's Clothing,Women's Upper Bodywear / Tops,Women's T-shirts / Tops,여성 티셔츠/탑스


In [14]:
y_tot = pd.merge(y_df, product, left_on = 'pd_c', right_on = 'pd_c', how = 'left');y_tot

,clnt_id,sess_id,sech_kwd,pd_c,clac_nm1,clac_nm2,clac_nm3,kor_clac_nm3
0,2,1,레쓰비,NaN,NaN,NaN,NaN,NaN
1,2,1,레쓰비,NaN,NaN,NaN,NaN,NaN
2,2,1,레쓰비,NaN,NaN,NaN,NaN,NaN
3,2,1,레쓰비,NaN,NaN,NaN,NaN,NaN
4,2,1,레쓰비,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
458173,72424,1,enqn,NaN,NaN,NaN,NaN,NaN
458174,72424,1,gotqks,NaN,NaN,NaN,NaN,NaN
458175,72424,1,계란,NaN,NaN,NaN,NaN,NaN
458176,72424,2,NaN,0960,Meats,Domestic Porks,Domestic Porks - Shoulder Butts,국내산 돼지고기 - 어깨 버트


#### 검색기록과 구매 카테고리 병합

In [15]:
y_clnt_id = y_tot.clnt_id.unique()

In [16]:
y_s = []
y_t = []
for i in tqdm.tqdm_notebook(y_clnt_id):
    a = y_tot.query('clnt_id == @i')
    y_s.append(a.sech_kwd.unique())
    y_t.append(a.kor_clac_nm3.unique())

In [17]:
y_s[0]

array(['레쓰비', '레쓰비캔커피', nan, '초밥', '목우촌', '연유', '팥', '팥빙수팥', '크래미',
       '크래미맛살', '다진소고기', '레쓰비175', '우유200', '우유200ml', '감자깡', '레쓰비ㅣ',
       '생수', '진라면', '옥수수스프', '옥수수수프', '옥수수', '수프'], dtype=object)

In [18]:
y_t[0]

array([nan, '커피 음료', '통조림 야채 식품', '크림과 농축유', '두부 튀김', '크랩 스틱스', '라멘스'],
      dtype=object)

In [19]:
clean_y_s = []
for i in range(0, len(y_s)):
    a=[x for x in y_s[i] if str(x) != 'nan']
    clean_y_s.append(a)

In [20]:
clean_y_t = []
for i in range(0, len(y_t)):
    a=[x for x in y_t[i] if str(x) != 'nan']
    clean_y_t.append(a)

In [21]:
y_st = []
for i in range(0, len(y_s)):
    y_st.append(list(clean_y_s[i])+list(clean_y_t[i]))

In [22]:
y_text = []    
for i in range(0, len(y_st)):
    a = y_st[i]
    text = ''
    for j in range(0, len(a)):
        text = text + a[j] + ' '
    y_text.append(text)

In [23]:
y_text

['레쓰비 레쓰비캔커피 초밥 목우촌 연유 팥 팥빙수팥 크래미 크래미맛살 다진소고기 레쓰비175 우유200 우유200ml 감자깡 레쓰비ㅣ 생수 진라면 옥수수스프 옥수수수프 옥수수 수프 커피 음료 통조림 야채 식품 크림과 농축유 두부 튀김 크랩 스틱스 라멘스 ',
 '옥수수 비비고 진라면 롯데샌드 양파링 고메핫도그 크리스피 현미찹쌀 찹쌀가루 찹쌀 식빵 우유 투게더아이스크림 파스퇴르우유 그래놀라 냉동 만두 파스타 누들 쿠키 프레시 밀크 물. 냉동 핫도그 칫솔 파스타 소스 국수 기능성 밀크 치즈 피시 케이크 라멘스 다른 한국 전통 과자 ',
 '생수 2L ',
 '코데즈컴바인여성속옷 코데즈컴바인 브라탑 코데즈컴바인 캐미솔 여성 브라탑 만다리나덕 슬링백 레이첼콕스 사만사타바사 토트백 사만사타바사 크로스백 쉬폰원피스 쉬즈미스원피스 지지피엑스 스커트 쉬폰밴딩팬츠 쉬폰밴딩스커트 와이드팬츠 쉬폰밴딩와이드팬츠 라코스테 원피스 티파니 원피스 베네통여성 베네통키즈 베네통키즈 원피스 라코스테 슬링백 GGPX원피스 아이더 스투키 스투키 필라 여성팬티 세트 여성팬티세트 휠라팬티세트 휠라 노와이어 팬티세트 레이첼콕스 앵클 휠라키즈 휠라키즈 아동화 레이첼콕스 가을 가을구두 호피스커트 나이스클랍 여성블라우스 리스트 블라우스 사만사타바사가방 레이첼콕스 구두 맨투맨 리스트 맨투맨 더아이잗 맨투맨 쉬즈미스 맨투맨 이앤시 맨투맨 이앤씨 맨투맨 마이클코어스가방 유아복 / 토들러 드레스 ',
 '콜라 핫도그 우유 찰진쌀 빵 초밥 김밥 고메핫도그 라면사리 사리 부대찌개 치킨텐더 오델로 카레 순살치킨 스파게티소스 스파게티면 진라면 치킨 식빵 모닝빵 삼립모닝빵 샌드위치 감자칩 허니버터암ㅎㄴ드 허니버터아몬드 대패삼겹살 삼겹살 코디 롤휴지 기능성 밀크 냉동 핫도그 파스타 소스 통조림 튜나스 인스턴트 커리 / 블랙 콩 소스 인스턴트 라이스 파스타 누들 콜라 냉동 후라이드 식품 밥 닭알 라멘스 감자 스낵 ',
 '매그제이스커트 나인스커트 올리브데올리브 스커트 온앤온스커트 미쏘 스커트 A라인스커트 가을블라우스 플라스

### Word2Vec 학습

In [24]:
wv = Word2Vec(size=20,
              min_count=1,
              workers=4,
              sentences=[simple_preprocess(p) for p in y_text]
             )

In [25]:
wv.wv['커피']

array([-2.5204456 ,  0.66836977,  1.510119  , -1.076312  ,  3.335336  ,
        1.5735838 , -0.88081   , -1.2559885 ,  3.7837186 ,  1.0433207 ,
        3.6049721 ,  2.3952644 , -1.9215472 , -0.6499587 , -0.30458927,
       -3.3093576 , -4.9569783 , -2.2705603 ,  0.08917051,  4.457526  ],
      dtype=float32)

In [26]:
wv.wv.most_similar('커피')

[('부드러운', 0.9352742433547974),
 ('음료수', 0.9261784553527832),
 ('콜라', 0.919846773147583),
 ('음료', 0.9096559286117554),
 ('믹스', 0.8925400972366333),
 ('믹스커피', 0.8851101994514465),
 ('스틱', 0.8782070279121399),
 ('주머니', 0.8757156133651733),
 ('과일주스', 0.8729631900787354),
 ('사과주', 0.8726745843887329)]

In [27]:
wv_words = sorted(set(wv.wv.vocab))

In [28]:
wvmat = wv.wv[wv_words]

In [29]:
wvmat.shape

(36118, 20)

#### 각 유저에 대한 대표 벡터 생성

In [30]:
pre_y_text = [simple_preprocess(p) for p in y_text];pre_y_text

[['레쓰비',
  '레쓰비캔커피',
  '초밥',
  '목우촌',
  '연유',
  '팥빙수팥',
  '크래미',
  '크래미맛살',
  '다진소고기',
  '레쓰비',
  '우유',
  '우유',
  'ml',
  '감자깡',
  '레쓰비ㅣ',
  '생수',
  '진라면',
  '옥수수스프',
  '옥수수수프',
  '옥수수',
  '수프',
  '커피',
  '음료',
  '통조림',
  '야채',
  '식품',
  '크림과',
  '농축유',
  '두부',
  '튀김',
  '크랩',
  '스틱스',
  '라멘스'],
 ['옥수수',
  '비비고',
  '진라면',
  '롯데샌드',
  '양파링',
  '고메핫도그',
  '크리스피',
  '현미찹쌀',
  '찹쌀가루',
  '찹쌀',
  '식빵',
  '우유',
  '투게더아이스크림',
  '파스퇴르우유',
  '그래놀라',
  '냉동',
  '만두',
  '파스타',
  '누들',
  '쿠키',
  '프레시',
  '밀크',
  '냉동',
  '핫도그',
  '칫솔',
  '파스타',
  '소스',
  '국수',
  '기능성',
  '밀크',
  '치즈',
  '피시',
  '케이크',
  '라멘스',
  '다른',
  '한국',
  '전통',
  '과자'],
 ['생수'],
 ['코데즈컴바인여성속옷',
  '코데즈컴바인',
  '브라탑',
  '코데즈컴바인',
  '캐미솔',
  '여성',
  '브라탑',
  '만다리나덕',
  '슬링백',
  '레이첼콕스',
  '사만사타바사',
  '토트백',
  '사만사타바사',
  '크로스백',
  '쉬폰원피스',
  '쉬즈미스원피스',
  '지지피엑스',
  '스커트',
  '쉬폰밴딩팬츠',
  '쉬폰밴딩스커트',
  '와이드팬츠',
  '쉬폰밴딩와이드팬츠',
  '라코스테',
  '원피스',
  '티파니',
  '원피스',
  '베네통여성',
  '베네통키즈',
  '베네통키즈',
  '원피스',
  '라코스테',
  '슬링백',
  'ggpx원피스',


검색과 구매한 키워드의 좌표를 평균내어 계산

In [31]:
y_wv = []
for i in range(0, len(pre_y_text)):
    a =  pre_y_text[i]
    b = np.zeros(20)
    for j in range(0, len(a)):
        b = b + wv.wv[a[j]]
        c = b/len(a)
    y_wv.append(c)

In [32]:
display(len(y_wv))
display(y_wv[0].shape)

10501

(20,)

### 구매없이 검색만 진행한 고객(동일 과정 진행)

In [33]:
n_clnt = online.query('clnt_id != @trans_clnt_id').sort_values(by = ['clnt_id','sess_id','hit_seq']).loc[:,['clnt_id', 'sess_id','sech_kwd']]

In [34]:
n_df = n_clnt.dropna();n_df

,clnt_id,sess_id,sech_kwd
3102890,1,1,과일선물세트
3136925,1,1,과일선물세트 백화점
3065714,1,1,과일바구니
3091872,1,2,초등가을잠바
3142610,1,2,초등가을점퍼
...,...,...,...
3090145,72428,3,구찌
3150794,72428,3,루이비통
3148589,72428,3,페레가모가방
3148588,72428,4,펜디가방


학습된 W2V의 voca에 없는 단어를 검색한 유저 제거

In [35]:
n_df = n_df.query('sech_kwd in @wv_words')

In [36]:
n_clnt_id = n_df.clnt_id.unique()

In [37]:
n_s = []
for i in tqdm.tqdm_notebook(n_clnt_id):
    a = n_df.query('clnt_id == @i')
    n_s.append(a.sech_kwd.unique())

In [38]:
n_text = []    
for i in range(0, len(n_s)):
    a = n_s[i]
    text = ''
    for j in range(0, len(a)):
        text = text + a[j] + ' '
    n_text.append(text)

In [39]:
n_text

['과일선물세트 과일바구니 노스페이스키즈 ',
 '블랙원피스 정장원피스 ',
 '보리쌀 보리 드립 카누미니 ',
 '레몬 고당도수박 수박 음성수박 ',
 '조셉앤스테이시 ',
 '애호박 놀부부대찌개 라이너 식빵 삼겹살 닭갈비 당면 오뚜기당면 ',
 '에어맥스 ',
 '샤인머스켓 대리석 ',
 '등산화 등산바지 남성여름등산바지 ',
 '아모레퍼시픽 ',
 '셔츠원피스 남성지갑 역시즌 반바지 반지갑 흰색티셔츠 반팔티 ',
 '포몽드 ',
 '슬라이스햄 식빵 ',
 '맥스봉 찌든때제거제 ',
 '밍크뮤 ',
 '언더웨어 ck언더웨어 ',
 '왕만두 명절세트 커피 제주청귤 ',
 '쉬폰블라우스 미니원피스 냉장고 상쾌환 마스크팩 갈치 la갈비 레이첼콕스 원피스 롱원피스 레이스원피스 ',
 '맥심화이트골드 아이스티 ',
 '린넨원피스 원피스 래쉬가드여성 배럴래쉬가드여성 아레나래쉬가드여성 레노마래쉬가드여성 책가방 아디다스책가방 알로봇 엠엘비키즈 블랙야크키즈 슬링백 ',
 '나무하나 ',
 '드롱기 ',
 '생수 우유 ',
 '나무하나플랫슈즈 ',
 '산과들에 ',
 '리빙박스 ',
 '트로피카나 ',
 '불가리쿠스 ',
 '블랙박스 ',
 '면도기 불고기 만두 ',
 '닭가슴살 ',
 '생수 요플레 코코팜 쿨피스 주스 에이스 ',
 '반에이크원피스 쪼리 버켄스탁 포멜카멜레샌들 알도샌들 저스트지니 ',
 '필라 휠라 ',
 '여성자켓 ',
 '퓨마 마블 다니엘에스떼 브이넥티셔츠 티셔츠 조르쥬레쉬 블라우스 테이트 야상 프릴블라우스 리본블라우스 ',
 '입생로랑가방 커밍스텝 ',
 '해피콜 ',
 '미니백 ',
 '모기약스프레이 ',
 '올젠 남성티셔츠 ',
 '롯데면세점 ',
 '에코버 ',
 '치즈스틱 두부 버터 비누 수프리모아메리카노 크레파스 ',
 '헤지스가방 ',
 '여성샌들 수박 다우니 누오보 쿤달 카밀핸드크림 카밀 ',
 '빈폴레이디스 보테가베네타가방 노스페이스키즈 젖병 ',
 '우유 생크림 파스타 연유 김밥김 단무지 시금치 당근 참치 깻잎 상하목장 요거트 앞다리 

In [40]:
len(n_text)

20753

In [41]:
pre_n_text = [simple_preprocess(p) for p in n_text];pre_n_text

[['과일선물세트', '과일바구니', '노스페이스키즈'],
 ['블랙원피스', '정장원피스'],
 ['보리쌀', '보리', '드립', '카누미니'],
 ['레몬', '고당도수박', '수박', '음성수박'],
 ['조셉앤스테이시'],
 ['애호박', '놀부부대찌개', '라이너', '식빵', '삼겹살', '닭갈비', '당면', '오뚜기당면'],
 ['에어맥스'],
 ['샤인머스켓', '대리석'],
 ['등산화', '등산바지', '남성여름등산바지'],
 ['아모레퍼시픽'],
 ['셔츠원피스', '남성지갑', '역시즌', '반바지', '반지갑', '흰색티셔츠', '반팔티'],
 ['포몽드'],
 ['슬라이스햄', '식빵'],
 ['맥스봉', '찌든때제거제'],
 ['밍크뮤'],
 ['언더웨어', 'ck언더웨어'],
 ['왕만두', '명절세트', '커피', '제주청귤'],
 ['쉬폰블라우스',
  '미니원피스',
  '냉장고',
  '상쾌환',
  '마스크팩',
  '갈치',
  'la갈비',
  '레이첼콕스',
  '원피스',
  '롱원피스',
  '레이스원피스'],
 ['맥심화이트골드', '아이스티'],
 ['린넨원피스',
  '원피스',
  '래쉬가드여성',
  '배럴래쉬가드여성',
  '아레나래쉬가드여성',
  '레노마래쉬가드여성',
  '책가방',
  '아디다스책가방',
  '알로봇',
  '엠엘비키즈',
  '블랙야크키즈',
  '슬링백'],
 ['나무하나'],
 ['드롱기'],
 ['생수', '우유'],
 ['나무하나플랫슈즈'],
 ['산과들에'],
 ['리빙박스'],
 ['트로피카나'],
 ['불가리쿠스'],
 ['블랙박스'],
 ['면도기', '불고기', '만두'],
 ['닭가슴살'],
 ['생수', '요플레', '코코팜', '쿨피스', '주스', '에이스'],
 ['반에이크원피스', '쪼리', '버켄스탁', '포멜카멜레샌들', '알도샌들', '저스트지니'],
 ['필라', '휠라'],
 ['여성자켓'],
 ['퓨마',
  '마블',
  '다니엘에스떼'

In [42]:
len(pre_n_text)

20753

In [43]:
n_wv= []
for i in range(0, len(pre_n_text)):
    a =  pre_n_text[i]
    b = np.zeros(20)
    for j in range(0, len(a)):
        b = b + wv.wv[a[j]]
        c = b/len(a)
    n_wv.append(c)

In [44]:
display(len(y_wv))
display(y_wv[0].shape)
display(len(n_wv))
display(n_wv[0].shape)

10501

(20,)

20753

(20,)

In [45]:
new_product = pd.concat([p_c2.reset_index(), keyword],axis=1);new_product

,level_0,index,pd_c,clac_nm1,clac_nm2,clac_nm3,clac_nm3,kor_clac_nm3
0,1,413,0414,Fashion Accessories,Casual Bags,Accessory Bags,Accessory Bags,액세서리 가방
1,2,14,0015,Automotive Products,Car Safety Equipment,Accident Prevention Equipment,Accident Prevention Equipment,사고 방지 장비
2,3,1338,1339,Stationary / Office Supplies,General Stationary / Office Supplies,Adhesive Tapes,Adhesive Tapes,접착 테이프
3,4,1463,1464,Tools / Safety Supplies,Improvement Supplies,Adhesives,Adhesives,접착제
4,5,61,0062,Bedding / Handicraft,Adults' Bedding,Adult's Bed Covers and Skirts,Adult's Bed Covers and Skirts,성인용 침대 커버와 스커트
...,...,...,...,...,...,...,...,...
1659,1660,1611,1612,Vegetables,Root Vegetables,Young Radishes,Young Radishes,어린 무
1660,1661,548,0549,Fruits,Domestic Fruits,Yuzus / Quinces,Yuzus / Quinces,유주스 / 왕자들
1661,1662,888,0889,Kitchenware,Kitchen Organization / Disposable Goods,Zipper Poly Bags / Plastic Bags,Zipper Poly Bags / Plastic Bags,지퍼 폴리 가방/플라스틱
1662,1663,1150,1151,Sea Foods,Seaweeds,kelp,kelp,다시마


### 코사인 유사도 계산 및 가장 유사한 고객의 구매내역 추출

In [ ]:
index_t = []
index_c = []
for i in tqdm.tqdm_notebook(range(0, len(n_wv))):
    a = i
    cos = []
    for j in range(0, len(y_wv)):
        cos.append(cosine_similarity(n_wv[a].reshape(1,-1),y_wv[j].reshape(1,-1)))
        
    cos_df = pd.DataFrame({
        'cos_sim': cos
    }).sort_values(by='cos_sim', ascending = False).reset_index()
    
    sim_c = cos_df.iloc[1,0]
    index_c.append(sim_c)
    
    b = clean_y_t[sim_c]
    sim_t = new_product.query('kor_clac_nm3 in @b').iloc[:,1].values
    index_t.append(sim_t)

In [ ]:
final = pd.DataFrame({
    'clnt_id': n_df.clnt_id.unique(),
    'trans': index_t
    'sim_id': index_c
})

In [ ]:
final.to_csv('final.csv', encoding = 'utf-8', index = False)